In [ ]:
from langchain_openai import ChatOpenAI
from langchain_core.messages import SystemMessage, HumanMessage

openai_api_base = (
    "http://vllm-gpt-oss-120b-svc.pai-custom-models.svc.cluster.local:8000/v1"
)
model_name = "/models/models/gpt-oss-120b"

llm = ChatOpenAI(
    openai_api_key="EMPTY",
    openai_api_base=openai_api_base,
    model_name=model_name,
    temperature=0.2,
    max_tokens=8000,
    extra_body={},
)

In [ ]:
import dataik
from tqdm.auto import tqdm
import pandas as pd, numpy as np
from elasticsearch.helpers import scan

from pydantic import BaseModel, Field
from dataiku import pandasutils as pdu
from elasticsearch import Elasticsearch
from langchain_elasticsearch import ElasticsearchStore
from langchain_elasticsearch import ElasticsearchRetriever
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.output_parsers import PydanticOutputParser

# 불러와야 할 것들
client = dataiku.api_client()
project = client.get_default_project()

emb = project.get_llm("허깅페이스어쩌구 bge-m3-korean")
emb = emb.as_langchain_embeddings()

es_connection = client.get_connection("con_esdb_defalut")
es_conn_info = es_connection.get_info()

index_id = "~~~"

# ES 연결 정보 가져옴
es_host = es_conn_info.get("params", {}).get("host")
es_port = es_conn_info.get("params", {}).get("port")
username = es_conn_info.get("params", {}).get("username")
password = es_conn_info.get("params", {}).get("password")

es_client = Elasticsearch(
    [{"host": es_host, "port": es_port, "scheme": "https"}],
    basic_auth=(username, password),
    verify_certs=False,
    ca_certs=None,
)

es_client_langchain = ElasticsearchStore(
    index_name=f"{project.project_key}_kb_{index_id}".lower(), es_connection=es_client
)

In [ ]:
# ES 연결 정보 가져옴
es_host = es_conn_info.get("params", {}).get("host")
es_port = es_conn_info.get("params", {}).get("port")
username = es_conn_info.get("params", {}).get("username")
password = es_conn_info.get("params", {}).get("password")

es_client = Elasticsearch(
    [{"host": es_host, "port": es_port, "scheme": "https"}],
    basic_auth=(username, password),
    verify_certs=False,
    ca_certs=None,
)

es_client_langchain = ElasticsearchStore(
    index_name=f"{project.project_key}_kb_{index_id}".lower(), es_connection=es_client
)

NameError: name 'es_conn_info' is not defined

In [ ]:
index_id = "~"
user_query = "할인되는 카드 있나요"
index_name = f"data_preprocess_kb_{index_id}".lower()


def reciprocal_rank_fusion(search_hits_1, search_hits_2, k: int = 60):
    rrf_scores = {}
    for rank, doc in enumerate(search_hits_1, start=1):
        doc_id = doc.metadata["_id"]
        if not doc_id in rrf_scores.keys():
            rrf_scores[doc_id] = 0
        rrf_scores[doc_id] += 1 / (k + rank)

    for rank, doc in enumerate(search_hits_2, start=1):
        doc_id = doc.metadata["_id"]
        if not doc_id in rrf_scores.keys():
            rrf_scores[doc_id] = 0
        rrf_scores[doc_id] += 1 / (k + rank)

    sorted_results = sorted(rrf_scores.items(), key=lambda x: x[1], reverse=True)

    return sorted_results


def dvec_query(search_query: str) -> dict:
    """
    Dense vector 검색 쿼리 구성
    """
    vector = emb.embed_query(search_query)
    return {
        "knn": {
            "field": "vector",
            "query_vector": vector,
            "k": 10,
            "num_candidates": 100,
        }
    }


def svec_query(search_query: str) -> dict:
    """
    Sparse vector 검색 쿼리 구성
    """
    return None


def kw_query(search_query: str) -> dict:
    """
    Keyword (BM25) 검색 쿼리 구성
    """
    return {
        "query": {
            # "size": 3,
            "match": {"text": {"query": user_query, "fuzziness": "AUTO"}}
        },
    }


def hybrid_query(search_query: str) -> dict:
    """
    추후 es license 들여오면 대체해서 사용
    """

    return {
        "retriever": {
            "rrf": {
                "retrievers": [
                    dvec_query(search_query),
                    {"standard": kw_query(search_query)},
                ],
                # "window_size" : 3
            }
        },
        "size": 3,
    }


dvec_retriever = ElasticsearchRetriever(
    es_client=es_client,
    index_name=index_name,
    body_func=dvec_query,
    content_field="text",
)

dvec_hits = dvec_retriever.invoke(user_query)

kw_retriever = ElasticsearchRetriever(
    es_client=es_client, index_name=index_name, body_func=kw_query, content_field="text"
)

kw_hits = kw_retriever.invoke(user_query)

hb_retriever = ElasticsearchRetriever(
    es_client=es_client,
    index_name=index_name,
    body_func=hybrid_query,
    content_field="text",
)

# 라이센스 비활성화 동안 사용
hybrid_scores = reciprocal_rank_fusion(kw_hits, dvec_hits)

# 라이센스 활성화 되면 사용
hb_hits = hb_retriever.invoke(user_query)
hb_hits

NameError: name 'ElasticsearchRetriever' is not defined

In [ ]:
index_id = "~~~"
index_name = "~~~"